## Order check of the turbine efficiency 
### comparison between correlated and uncorrelated sets of data
In this notebook the comparison between turbine efficiency obtained using correlated or uncorrelated temperatures and pressures will be done. To this aim the method $\textit{computeCoefficient}$ of the class $\textit{Polyint}$ has been used.

### Generation of the Basis
In the first part of the script a Basis has been contructed using four uniform distributions; the central values has been adapted to ensure that they corresponds to the mean values of temperatures and pressures.
- Question 1: Is it necessary? Is there a conceptual error?


In [ ]:
import numpy as np
from equadratures import *

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm

#----------------------------------------------------------------#
#  Polyint class instance.

myBasis   = Basis('Tensor grid')
uni_1  = Parameter(order=5, distribution='uniform', upper=810, lower =790)
uni_2  = Parameter(order=5, distribution='uniform', upper=710, lower =690)
uni_3  = Parameter(order=5, distribution='uniform', upper=9*10**5, lower=11*10**5)
uni_4  = Parameter(order=5, distribution='uniform', upper=4*10**5, lower=6*10**5)
myPolyint = Polyint([uni_1, uni_2, uni_3, uni_4], myBasis)

### Definition of the Efficiency function
The Turbine Efficiency function has been specified as reported in the lines below: 
- (PART 1) The input $x$ rappresents an array containing four magnitudes (Temperatures T01, T02 and Pressures P01, P02), so it specifies the problem dimension.
- (PART 2) The mean values of each magnitude has been set because four different truncated-gaussian distributions have to be created for the Nataf class instance; the correlation matrix $R$ has been defined in the end of this part.
- (PART 3) This part contains the lines that will be used only in the case in which the correlated points have to used for the calculation of the turbine efficiency.
- (PART 4) In the last part the general formula of the turbine efficiency has been specified with its return statement.

In [ ]:
def efficiency(x):
    # (PART 1)
    t1 = x[0]
    t2 = x[1]
    p1 = x[2]
    p2 = x[3]
    gamma =1.4

    """ (PART 2)
        construction of the correlated points: 
        temperatures and pressures
    """
    T01 = 800.
    T02 = 700.
    P01 = 10*10**5
    P02 = 5*10**5
    
    distr_t1 = Parameter(distribution='truncated-gaussian', order=5, shape_parameter_A = T01, shape_parameter_B = 1., upper = T01*1.1, lower = T01*.9)
    distr_t2 = Parameter(distribution='truncated-gaussian', order=5, shape_parameter_A = T02, shape_parameter_B = 1., upper = T02*1.1, lower = T02*.9)
    distr_p1 = Parameter(distribution='truncated-gaussian', order=5, shape_parameter_A = P01, shape_parameter_B = 1., upper = P01*1.1, lower = P01*.9)
    distr_p2 = Parameter(distribution='truncated-gaussian', order=5, shape_parameter_A = P02, shape_parameter_B = 1., upper = P02*1.1, lower = P02*.9)
    
    # definition of the correlation matrix
    R = np.matrix([[1., .3, 0., 0.],
                   [.3, 1., 0., 0.],
                   [0., 0., 1., .6],
                   [0., 0., .6, 1.]])
    
    # (PART 3)
    
    ##------------------------------------------------------------------------------#
    ## the lines from 58 to 70 have been commented in the case of uncorrelad 
    ## temperatures and pressures
    ##------------------------------------------------------------------------------#
    ## definition of a nataf object
    #nataf_obj = Nataf([distr_t1, distr_t2, distr_p1, distr_p2], R) 

    ## arrays of samples
    #unco_pnts = np.matrix([t1, t2, p1, p2])
    #corr_pnts = nataf_obj.U2C(unco_pnts) 

    ## definition of the two sets that will be used 
    ## and turbine efficiency

    #t1 = corr_pnts[0,0]
    #t2 = corr_pnts[0,1]
    #p1 = corr_pnts[0,2]
    #p2 = corr_pnts[0,3]
    ##------------------------------------------------------------------------------#
    
    #(PART 4)
    eta = (t1 -t2)/(t1* (1- (p2/p1)**((gamma-1.)/gamma)))

    return eta

    

Finally the method $\textit{computeCoefficients}$ has been used to define the polynomial coefficients.

In [ ]:
myPolyint.computeCoefficients(efficiency)

In the lines below the first two coefficients (related to the Temperatures) have been plotted (the example reported in the EQ tutorials has been used to this aim).
- Question: The two plotted coefficients are related to the first two magnitudes, so the temperatures: is it right?

In [ ]:
#--------------------------------------------------------------#
# from EQ documentation: comparison among coefficients
#--------------------------------------------------------------#
x,y,z, max_order = utils.twoDgrid(myPolyint.coefficients, myPolyint.basis.elements)
G = np.log10(np.abs(z))
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
cax = plt.scatter(x,y,s=480, marker='o', c=G, cmap='jet', alpha=.8)
plt.xlim(-.5, max_order)
plt.ylim(-.5, max_order)
adjust_spines(ax, ['left', 'bottom'])
ax.set_axisbelow(True)
plt.xlabel('coeff 1')
plt.ylabel('coeff 2')
plt.title("Uncorrelated Temp.and Pres.")
cbar = plt.colorbar(extend ='neither', spacing='proportional', orientation='vertical', shrink=.8, format="%.0f")
cbar.ax.tick_params(labelsize=13)
plt.savefig('Coefficients_uncorrelated1.png', dpi=200, bbox_inches='tight')
plt.close()
#print 'the coefficients are:', myPolyint.coefficients
#print 'the size of coefficients are:', (myPolyint.coefficients).shape

### Plots of the results 
The figure $\textit{Coefficients_ uncorrelated1.png}$ reports the results in the case in which temperatures and pressures are not correlated, while $\textit{Coefficients_ correlated1.png}$ is rappresentative of the results for the correlated data set of points.